In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
# FP1

In [285]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.fp1_lap_time`
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/3752184278.py:9: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [286]:
race="British Grand Prix"

In [287]:
df_bq=df_bq2[(df_bq2["GP"]==race)&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/2559496312.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
185,0 days 00:16:35.989000,ALB,23,NaT,1,1.0,0 days 00:14:40.223000,0 days 00:16:29.915000,NaT,0 days 00:00:52.249000,0 days 00:00:30.080000,NaT,0 days 00:16:06.048000,0 days 00:16:35.989000,239.0,222.0,NaN,239.0,False,HARD,1.0,True,Williams,0 days 00:14:40.223000,2024-07-05 11:31:16.834,1,NaN,False,,False,False,2024,British Grand Prix
186,0 days 00:31:43.615000,ALB,23,NaT,2,2.0,0 days 00:29:55.984000,NaT,NaT,0 days 00:00:41.768000,0 days 00:00:31.616000,NaT,0 days 00:31:12.267000,0 days 00:31:43.697000,280.0,239.0,245.0,293.0,False,HARD,2.0,False,Williams,0 days 00:29:55.984000,2024-07-05 11:46:32.595,1,NaN,False,,False,False,2024,British Grand Prix
22,0 days 00:33:15.391000,ALB,23,0 days 00:01:31.776000,3,2.0,NaT,NaT,0 days 00:00:29.120000,0 days 00:00:37.548000,0 days 00:00:25.108000,0 days 00:32:12.735000,0 days 00:32:50.283000,0 days 00:33:15.391000,313.0,259.0,246.0,321.0,True,HARD,3.0,False,Williams,0 days 00:31:43.615000,2024-07-05 11:48:20.226,1,NaN,False,,False,True,2024,British Grand Prix
23,0 days 00:34:45.494000,ALB,23,0 days 00:01:30.103000,4,2.0,NaT,NaT,0 days 00:00:28.711000,0 days 00:00:36.894000,0 days 00:00:24.498000,0 days 00:33:44.102000,0 days 00:34:20.996000,0 days 00:34:45.494000,314.0,260.0,248.0,324.0,True,HARD,4.0,False,Williams,0 days 00:33:15.391000,2024-07-05 11:49:52.002,1,NaN,False,,False,True,2024,British Grand Prix
187,0 days 00:36:46.983000,ALB,23,0 days 00:02:01.489000,5,2.0,NaT,NaT,0 days 00:00:40.254000,0 days 00:00:53.043000,0 days 00:00:28.192000,0 days 00:35:25.748000,0 days 00:36:18.791000,0 days 00:36:46.983000,210.0,197.0,249.0,283.0,False,HARD,5.0,False,Williams,0 days 00:34:45.494000,2024-07-05 11:51:22.105,1,NaN,False,,False,True,2024,British Grand Prix


In [288]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [289]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [290]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [291]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
185,ALB,1,0.0,52.249,30.080,0,0.0,HARD,2024,British Grand Prix,0.0,0.0,18.0,7.0
126,ALO,1,0.0,41.351,34.078,0,0.0,HARD,2024,British Grand Prix,0.0,0.0,1.0,17.0
340,BEA,1,0.0,46.565,28.759,0,0.0,HARD,2024,British Grand Prix,0.0,0.0,12.0,6.0
414,BOT,1,0.0,42.091,28.276,0,0.0,MEDIUM,2024,British Grand Prix,0.0,0.0,4.0,4.0
322,COL,1,0.0,42.386,27.155,0,0.0,HARD,2024,British Grand Prix,0.0,0.0,5.0,1.0


In [292]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [293]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,British Grand Prix,2024,2,00:01:29.906000,89.906,VER,HARD,28.837,VER,HARD,36.496,VER,HARD,24.573,VER,HARD
1,British Grand Prix,2024,3,00:01:29.864000,89.864,TSU,HARD,28.666,TSU,HARD,36.520,PIA,HARD,24.465,RIC,HARD
2,British Grand Prix,2024,4,00:01:28.888000,88.888,RUS,HARD,28.507,RUS,HARD,36.145,VER,HARD,24.160,RUS,HARD
3,British Grand Prix,2024,5,00:01:29.057000,89.057,LEC,HARD,28.446,ALO,HARD,36.217,LEC,HARD,24.248,LEC,HARD
4,British Grand Prix,2024,6,00:01:29.047000,89.047,STR,MEDIUM,28.467,STR,MEDIUM,36.172,STR,MEDIUM,24.334,ALB,HARD


In [294]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,British Grand Prix,2024,2,00:01:29.906000,89.906,VER,HARD,28.837,VER,HARD,36.496,VER,HARD,24.573,VER,HARD,#3860C6
1,British Grand Prix,2024,3,00:01:29.864000,89.864,TSU,HARD,28.666,TSU,HARD,36.520,PIA,HARD,24.465,RIC,HARD,#6692FF
2,British Grand Prix,2024,4,00:01:28.888000,88.888,RUS,HARD,28.507,RUS,HARD,36.145,VER,HARD,24.160,RUS,HARD,#27F4D2
3,British Grand Prix,2024,5,00:01:29.057000,89.057,LEC,HARD,28.446,ALO,HARD,36.217,LEC,HARD,24.248,LEC,HARD,#E80020
4,British Grand Prix,2024,6,00:01:29.047000,89.047,STR,MEDIUM,28.467,STR,MEDIUM,36.172,STR,MEDIUM,24.334,ALB,HARD,#229971


In [295]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
14,British Grand Prix,2024,15,00:01:27.420000,87.420,NOR,SOFT,27.933,NOR,SOFT,35.531,NOR,SOFT,23.956,NOR,SOFT,#FF8000
12,British Grand Prix,2024,13,00:01:27.554000,87.554,STR,SOFT,27.829,STR,SOFT,35.589,STR,SOFT,24.136,STR,SOFT,#229971
13,British Grand Prix,2024,14,00:01:27.631000,87.631,PIA,SOFT,28.085,PIA,SOFT,35.478,RUS,MEDIUM,23.938,RUS,MEDIUM,#FF8000
15,British Grand Prix,2024,16,00:01:27.697000,87.697,PIA,SOFT,28.125,PIA,SOFT,35.472,PIA,SOFT,24.028,SAI,MEDIUM,#FF8000
11,British Grand Prix,2024,12,00:01:27.729000,87.729,VER,MEDIUM,28.177,LEC,MEDIUM,35.433,VER,MEDIUM,23.919,RUS,MEDIUM,#3860C6


In [296]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=15)
    ))
 )

fig2.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": LapTime_ordered.T.values
                                if c == "All"
                                else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig2.update_layout(
    title="FP1 Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=16,
        color="Black"
    ),
    height=1600,
    title_font_family="PT Sans Narrow"
)

fig2.show()

In [297]:
fig2.write_html("{} {} FP1 Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))

In [298]:
#FP2

In [299]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.fp2_lap_time`
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/1418048944.py:9: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [300]:
df_bq=df_bq2[(df_bq2["GP"]==race)&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/2559496312.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
17982,0 days 00:15:48.338000,ALB,23,NaT,1,1.0,0 days 00:13:50.257000,NaT,NaT,0 days 00:00:44.424000,0 days 00:00:31.343000,NaT,0 days 00:15:17.215000,0 days 00:15:48.527000,296.0,236.0,249.0,285.0,False,MEDIUM,1.0,True,Williams,0 days 00:13:50.257000,2024-07-05 15:00:17.464,1,NaN,False,,False,False,2024,British Grand Prix
17660,0 days 00:17:17.930000,ALB,23,0 days 00:01:29.592000,2,1.0,NaT,NaT,0 days 00:00:28.459000,0 days 00:00:36.586000,0 days 00:00:24.547000,0 days 00:16:16.797000,0 days 00:16:53.383000,0 days 00:17:17.930000,312.0,242.0,248.0,323.0,True,MEDIUM,2.0,True,Williams,0 days 00:15:48.338000,2024-07-05 15:02:15.545,1,NaN,False,,False,True,2024,British Grand Prix
17809,0 days 00:19:35.514000,ALB,23,0 days 00:02:17.584000,3,1.0,NaT,NaT,0 days 00:00:43.146000,0 days 00:00:59.976000,0 days 00:00:34.462000,0 days 00:18:01.076000,0 days 00:19:01.052000,0 days 00:19:35.514000,195.0,157.0,249.0,196.0,False,MEDIUM,3.0,True,Williams,0 days 00:17:17.930000,2024-07-05 15:03:45.137,1,NaN,False,,False,True,2024,British Grand Prix
17810,0 days 00:21:36.126000,ALB,23,0 days 00:02:00.612000,4,1.0,NaT,NaT,0 days 00:00:28.589000,0 days 00:00:56.874000,0 days 00:00:35.149000,0 days 00:20:04.103000,0 days 00:21:00.977000,0 days 00:21:36.126000,313.0,186.0,239.0,263.0,False,MEDIUM,4.0,True,Williams,0 days 00:19:35.514000,2024-07-05 15:06:02.721,1,NaN,False,,False,True,2024,British Grand Prix
17811,0 days 00:23:52.988000,ALB,23,0 days 00:02:16.862000,5,1.0,NaT,NaT,0 days 00:00:42.036000,0 days 00:00:54.488000,0 days 00:00:40.338000,0 days 00:22:18.162000,0 days 00:23:12.650000,0 days 00:23:52.988000,259.0,222.0,250.0,302.0,False,MEDIUM,5.0,True,Williams,0 days 00:21:36.126000,2024-07-05 15:08:03.333,1,NaN,False,,False,True,2024,British Grand Prix


In [301]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [302]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [303]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [304]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
17982,ALB,1,0.0,44.424,31.343,0,0.0,MEDIUM,2024,British Grand Prix,0.0,0.0,14.0,5.0
17961,ALO,1,0.0,42.879,35.268,0,0.0,MEDIUM,2024,British Grand Prix,0.0,0.0,9.0,10.0
18014,BOT,1,0.0,40.768,33.023,0,0.0,HARD,2024,British Grand Prix,0.0,0.0,3.0,8.0
17954,GAS,1,0.0,45.610,32.939,0,0.0,MEDIUM,2024,British Grand Prix,0.0,0.0,17.0,7.0
18002,HAM,1,0.0,41.499,44.543,0,0.0,MEDIUM,2024,British Grand Prix,0.0,0.0,5.0,18.0


In [305]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [306]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,British Grand Prix,2024,2,00:01:27.831000,87.831,VER,MEDIUM,28.184,VER,MEDIUM,35.623,VER,MEDIUM,24.024,VER,MEDIUM
1,British Grand Prix,2024,3,00:01:29.390000,89.390,HUL,HARD,28.844,HUL,HARD,36.156,HUL,HARD,24.390,HUL,HARD
2,British Grand Prix,2024,4,00:01:28.101000,88.101,HAM,MEDIUM,28.313,GAS,MEDIUM,35.801,HAM,MEDIUM,23.963,PIA,MEDIUM
3,British Grand Prix,2024,5,00:01:27.233000,87.233,VER,SOFT,28.179,VER,SOFT,35.177,VER,SOFT,23.877,VER,SOFT
4,British Grand Prix,2024,6,00:01:28.589000,88.589,TSU,MEDIUM,28.335,RIC,MEDIUM,35.858,TSU,MEDIUM,24.190,ALO,MEDIUM


In [307]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,British Grand Prix,2024,2,00:01:27.831000,87.831,VER,MEDIUM,28.184,VER,MEDIUM,35.623,VER,MEDIUM,24.024,VER,MEDIUM,#3860C6
1,British Grand Prix,2024,3,00:01:29.390000,89.390,HUL,HARD,28.844,HUL,HARD,36.156,HUL,HARD,24.390,HUL,HARD,#B6BABD
2,British Grand Prix,2024,4,00:01:28.101000,88.101,HAM,MEDIUM,28.313,GAS,MEDIUM,35.801,HAM,MEDIUM,23.963,PIA,MEDIUM,#27F4D2
3,British Grand Prix,2024,5,00:01:27.233000,87.233,VER,SOFT,28.179,VER,SOFT,35.177,VER,SOFT,23.877,VER,SOFT,#3860C6
4,British Grand Prix,2024,6,00:01:28.589000,88.589,TSU,MEDIUM,28.335,RIC,MEDIUM,35.858,TSU,MEDIUM,24.190,ALO,MEDIUM,#6692FF


In [308]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
10,British Grand Prix,2024,12,00:01:26.549000,86.549,NOR,SOFT,27.712,NOR,SOFT,35.060,NOR,SOFT,23.777,NOR,SOFT,#FF8000
8,British Grand Prix,2024,10,00:01:26.880000,86.880,PIA,SOFT,27.877,PIA,SOFT,35.121,PIA,SOFT,23.882,PIA,SOFT,#FF8000
12,British Grand Prix,2024,14,00:01:26.983000,86.983,PER,SOFT,28.031,PER,SOFT,35.112,PER,SOFT,23.840,PER,SOFT,#3860C6
9,British Grand Prix,2024,11,00:01:26.990000,86.990,HUL,SOFT,27.813,HUL,SOFT,35.226,PER,SOFT,23.816,HAM,SOFT,#B6BABD
3,British Grand Prix,2024,5,00:01:27.233000,87.233,VER,SOFT,28.179,VER,SOFT,35.177,VER,SOFT,23.877,VER,SOFT,#3860C6


In [309]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=15)
    ))
 )

fig2.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": LapTime_ordered.T.values
                                if c == "All"
                                else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig2.update_layout(
    title="FP2 Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=16,
        color="Black"
    ),
    height=1600,
    title_font_family="PT Sans Narrow"
)

fig2.show()

In [310]:
fig2.write_html("{} {} FP2 Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))

In [311]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.fp3_lap_time`
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/3201125824.py:9: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [312]:
df_bq=df_bq2[(df_bq2["GP"]==race)&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5214/2559496312.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
10564,0 days 00:18:18.451000,ALB,23,NaT,1,1.0,0 days 00:16:16.415000,NaT,NaT,0 days 00:00:49.654000,0 days 00:00:34.351000,NaT,0 days 00:17:44.276000,0 days 00:18:18.451000,241.0,214.0,181.0,273.0,False,INTERMEDIATE,1.0,True,Williams,0 days 00:16:16.415000,2024-07-06 10:31:20.077,1,NaN,False,,False,False,2024,British Grand Prix
10570,0 days 00:20:12.552000,ALB,23,0 days 00:01:54.101000,2,1.0,NaT,0 days 00:20:06.054000,0 days 00:00:36.159000,0 days 00:00:47.424000,0 days 00:00:30.518000,0 days 00:18:54.610000,0 days 00:19:42.034000,0 days 00:20:12.552000,279.0,221.0,NaN,285.0,False,INTERMEDIATE,2.0,True,Williams,0 days 00:18:18.451000,2024-07-06 10:33:22.113,1,NaN,False,,False,False,2024,British Grand Prix
12324,0 days 00:29:31.606000,ALB,23,NaT,3,2.0,0 days 00:20:38.711000,0 days 00:24:46.962000,0 days 00:01:02.122000,0 days 00:00:49.401000,NaT,0 days 00:21:14.790000,0 days 00:22:04.320000,NaT,267.0,221.0,NaN,274.0,False,INTERMEDIATE,3.0,False,Williams,0 days 00:20:12.552000,2024-07-06 10:35:16.214,125,NaN,False,,False,False,2024,British Grand Prix
12280,0 days 00:31:22.756000,ALB,23,NaT,4,3.0,0 days 00:29:31.606000,NaT,NaT,0 days 00:00:46.620000,0 days 00:00:31.642000,NaT,0 days 00:30:51.389000,0 days 00:31:22.919000,270.0,226.0,196.0,287.0,False,INTERMEDIATE,3.0,False,Williams,0 days 00:29:31.606000,2024-07-06 10:44:35.268,1,NaN,False,,False,False,2024,British Grand Prix
8500,0 days 00:33:11.935000,ALB,23,0 days 00:01:49.179000,5,3.0,NaT,NaT,0 days 00:00:34.719000,0 days 00:00:44.608000,0 days 00:00:29.852000,0 days 00:31:57.475000,0 days 00:32:42.083000,0 days 00:33:11.935000,288.0,232.0,202.0,293.0,True,INTERMEDIATE,4.0,False,Williams,0 days 00:31:22.756000,2024-07-06 10:46:26.418,12,NaN,False,,False,True,2024,British Grand Prix


In [313]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [314]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [315]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [316]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
10564,ALB,1,0.0,49.654,34.351,0,0.0,INTERMEDIATE,2024,British Grand Prix,0.0,0.0,14.0,14.0
10561,ALO,1,0.0,46.943,34.120,0,0.0,INTERMEDIATE,2024,British Grand Prix,0.0,0.0,2.0,12.0
10568,BOT,1,0.0,49.229,33.033,0,0.0,INTERMEDIATE,2024,British Grand Prix,0.0,0.0,12.0,5.0
10560,GAS,1,0.0,50.628,34.234,0,0.0,INTERMEDIATE,2024,British Grand Prix,0.0,0.0,17.0,13.0
12297,HAM,1,0.0,47.946,34.014,0,0.0,INTERMEDIATE,2024,British Grand Prix,0.0,0.0,8.0,11.0


In [317]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [318]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,British Grand Prix,2024,2,00:01:48.364000,108.364,LEC,INTERMEDIATE,33.903,VER,INTERMEDIATE,42.898,VER,INTERMEDIATE,29.272,HUL,INTERMEDIATE
1,British Grand Prix,2024,3,00:01:47.080000,107.080,TSU,INTERMEDIATE,33.295,ALO,INTERMEDIATE,42.472,ALO,INTERMEDIATE,29.610,TSU,INTERMEDIATE
2,British Grand Prix,2024,4,00:01:43.697000,103.697,NOR,INTERMEDIATE,32.722,TSU,INTERMEDIATE,41.934,NOR,INTERMEDIATE,28.454,NOR,INTERMEDIATE
3,British Grand Prix,2024,5,00:01:41.087000,101.087,RUS,INTERMEDIATE,32.326,RUS,INTERMEDIATE,40.961,RUS,INTERMEDIATE,27.783,NOR,INTERMEDIATE
4,British Grand Prix,2024,6,00:01:40.328000,100.328,SAI,INTERMEDIATE,31.934,SAI,INTERMEDIATE,40.465,SAI,INTERMEDIATE,27.929,SAI,INTERMEDIATE


In [319]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,British Grand Prix,2024,2,00:01:48.364000,108.364,LEC,INTERMEDIATE,33.903,VER,INTERMEDIATE,42.898,VER,INTERMEDIATE,29.272,HUL,INTERMEDIATE,#E80020
1,British Grand Prix,2024,3,00:01:47.080000,107.080,TSU,INTERMEDIATE,33.295,ALO,INTERMEDIATE,42.472,ALO,INTERMEDIATE,29.610,TSU,INTERMEDIATE,#6692FF
2,British Grand Prix,2024,4,00:01:43.697000,103.697,NOR,INTERMEDIATE,32.722,TSU,INTERMEDIATE,41.934,NOR,INTERMEDIATE,28.454,NOR,INTERMEDIATE,#FF8000
3,British Grand Prix,2024,5,00:01:41.087000,101.087,RUS,INTERMEDIATE,32.326,RUS,INTERMEDIATE,40.961,RUS,INTERMEDIATE,27.783,NOR,INTERMEDIATE,#27F4D2
4,British Grand Prix,2024,6,00:01:40.328000,100.328,SAI,INTERMEDIATE,31.934,SAI,INTERMEDIATE,40.465,SAI,INTERMEDIATE,27.929,SAI,INTERMEDIATE,#E80020


In [320]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
16,British Grand Prix,2024,18,00:01:36.654000,96.654,PIA,INTERMEDIATE,31.344,RUS,INTERMEDIATE,39.532,PIA,INTERMEDIATE,25.716,PIA,INTERMEDIATE,#FF8000
9,British Grand Prix,2024,11,00:01:37.529000,97.529,RUS,INTERMEDIATE,31.034,RUS,INTERMEDIATE,39.500,HAM,INTERMEDIATE,26.869,RUS,INTERMEDIATE,#27F4D2
11,British Grand Prix,2024,13,00:01:37.564000,97.564,HAM,INTERMEDIATE,31.105,HAM,INTERMEDIATE,39.296,HAM,INTERMEDIATE,27.163,HAM,INTERMEDIATE,#27F4D2
13,British Grand Prix,2024,15,00:01:37.714000,97.714,NOR,INTERMEDIATE,31.382,NOR,INTERMEDIATE,39.314,NOR,INTERMEDIATE,27.018,NOR,INTERMEDIATE,#FF8000
12,British Grand Prix,2024,14,00:01:37.812000,97.812,NOR,INTERMEDIATE,31.256,SAI,INTERMEDIATE,39.327,SAI,INTERMEDIATE,26.320,HAM,INTERMEDIATE,#FF8000


In [321]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=15)
    ))
 )

fig2.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": LapTime_ordered.T.values
                                if c == "All"
                                else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig2.update_layout(
    title="FP3 Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=16,
        color="Black"
    ),
    height=1600,
    title_font_family="PT Sans Narrow"
)

fig2.show()

In [322]:
fig2.write_html("{} {} FP3 Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))